In [1]:
import sys
sys.path.append("../")
# import time
# time.sleep(7200)

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss

ModelsPerformance = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    hamLoss = hamming_loss(test_labels, predictions)
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    ModelsPerformance[modelName] = micro_f1

In [3]:
config = {
    "pretrained_model": "bert-base-uncased",
    #emilyalsentzer/Bio_ClinicalBERT ../resources/bert/biobert_v1.1_pubmed/
    "tokenizer": "bert-base-uncased",
    "max_seq_length": 256,
    "batch_size": 6,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "0,1,2,3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 90,
    "is_multilabel": True,
    "valid_metric": "micro_f1",
    "model_save_dir": "../checkpoints/bert_cased_256_bio_26k_20200908/",
    "patience": 4,
}

In [3]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [5]:
import torch
import numpy as np
def int2label(label):
    l_n = len(label)
    one_hot = np.array([0]*len(label))
    for i,c in enumerate(label):
        if c == '1':
            one_hot[i]=1
    return list(one_hot)
int2label("00001100")

[0, 0, 0, 0, 1, 1, 0, 0]

In [6]:
import pandas as pd
train_set = pd.read_csv("../datasets/Reuters/train.tsv", sep='\t', header=None)
train_set.rename(columns={1:"docs"},inplace=True)
train_set.loc[:,"labels"]=train_set[0].apply(int2label)
train_set.head()

,0,docs,labels
0,0000000000000000000000000000000000000000000000...,U.S. APPEARS TO TOLERATE FURTHER DLR DECLINE. ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0000000000000000000000000000000000000000000000...,Sumita says Bank of Japan will intervene if ne...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0000000000000000000000000000000000000000000000...,FIRS 87/88 EC BEET SUGAR ESTIMATE UNCHANGED 12...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0000000000000000000001000000000000000000000000...,UNICORP AMERICAN CORP <UAC> 4TH QTR NET. Shr 1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0000000000000000000000000000000000000000000000...,BRITISH MINISTER CRITICISES PROPOSED EC OILS T...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
dev_set = pd.read_csv("../datasets/Reuters/dev.tsv", sep='\t', header=None)
dev_set.rename(columns={1:"docs"},inplace=True)
dev_set.loc[:,"labels"]=dev_set[0].apply(int2label)
dev_set.head()

,0,docs,labels
0,0000000000000000000001000000000000000000000000...,<CANADIAN IMPERIAL BANK OF COMMERCE> 1ST QTR. ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1000000000000000000000000000000000000000000000...,REUTERS TO BUY I P SHARP OF CANADA. Reuters Ho...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0000000000000000000001000000000000000000000000...,KIRIN BREWERY CO LTD <KNBW.T> YEAR TO JANUARY ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0000000000000000000001000000000000000000000000...,HONGKONG BANK OF CANADA 1ST QTR JAN 31 NET. Ne...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0000000000000000000001000000000000000000000000...,CENTRAL CAPITAL PLANS THREE-FOR-TWO STOCK SPLI...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
test_set = pd.read_csv("../datasets/Reuters/test.tsv", sep='\t', header=None)
test_set.rename(columns={1:"docs"},inplace=True)
test_set.loc[:,"labels"]=test_set[0].apply(int2label)
test_set.head()

,0,docs,labels
0,0000000000000000000000000000000000000000000000...,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0000000000000000000000000010000000000000000000...,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0000000000000000010000000000000000000000000000...,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0000000000001000000000000010000000000000000000...,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER. Th...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,0000000000000000000000000000000000000000000000...,INDONESIA SEES CPO PRICE RISING SHARPLY. Indon...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vectorizer = TfidfVectorizer()
vectorised_train_documents = vectorizer.fit_transform(train_set["docs"])
vectorised_test_documents = vectorizer.transform(test_set["docs"])
train_labels = [label for label in train_set["labels"].values]
test_labels = [label for label in test_set["labels"].values]

In [22]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(vectorised_train_documents, train_labels)

svmPreds = svmClassifier.predict(vectorised_test_documents)
metricsReport("SVC Sq. Hinge Loss", test_labels, svmPreds)

/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.7970
Hamming Loss: 0.0036
Precision:
  - Macro: 0.5674
  - Micro: 0.9479
Recall:
  - Macro: 0.3238
  - Micro: 0.7780
F1-measure:
  - Macro: 0.3934
  - Micro: 0.8546


In [24]:
from sklearn import metrics

metrics.accuracy_score(test_labels, svmPreds)

0.7969526333222922

In [9]:
# train_set = data_df[data_df["flag"]=="train"]
# dev_set = data_df[data_df["flag"]=="valid"]
# test_set = data_df[data_df["flag"]=="test"]

In [9]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(5827, 1943, 3019)

In [10]:
cls.train(train_set,dev_set)


Execute [train_an_epoch] method costing 106305.80 ms
[Epoch 0] loss: 707.013686478138


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Execute [eval] method costing 35488.77 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+-----------+
|    | metrics         |    values |
|----+-----------------+-----------|
|  0 | accuracy        | 0         |
|  1 | macro_precision | 0.0123161 |
|  2 | macro_recall    | 0.426508  |
|  3 | macro_f1        | 0.0151088 |
|  4 | micro_precision | 0.0132213 |
|  5 | micro_recall    | 0.519464  |
|  6 | micro_f1        | 0.0257862 |
+----+-----------------+-----------+
--------------------------------------------------------------------------------


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Execute [train_an_epoch] method costing 106040.85 ms
[Epoch 1] loss: 707.008339881897


...


Execute [train_an_epoch] method costing 106000.64 ms
[Epoch 2] loss: 706.7762475609779



Execute [eval] method costing 35482.27 ms



Execute [train_an_epoch] method costing 106062.77 ms
[Epoch 3] loss: 706.7258413434029



Execute [eval] method costing 35517.49 ms


KeyboardInterrupt: 

In [ ]:
result = cls.test(test_set)
result

In [ ]:
import csv
with open('../results/bert_cased_256_bio_26k_20200908_test.csv', 'w') as f:
    for key in result.keys():
        f.write("%s,%s\n"%(key,result[key]))

In [ ]:
result